# Network Genius: On-demand observability of Network data using Langchain SQL Agent, Athena and Amazon Bedrock 

**Info.**  
The current lab1 workshop is part of a solution called Network Genius. Network Genius is a multi-scenario solution that includes features like network Configuration generation, multi-db observability, etc. The query generation feature is part of Network_Genius_v0. 

In this notebook, we will create a conversational agent that allows you to ask questions about your network data. In a couple of seconds, as a network operations team, you will be able to drill-down, assess the health-check of your network resources, and build on-the-fly network Key Performance Indicators without predefining them.

In this lab, you will be doing a drill down process for network failures as well as for the impacted customers. 

The network drill down process follows these steps:

- Identify how many active users are in California

- assessing what are the reasons of failures in California and expecting that the model will list and explain them in term of drop calls, white communications, etc. 

- from those reasons of failure in California, we select the code 38 which is about Network out of order and ask about the associated top 10 cells.

- Now that we have the cells IDs with release 38, we want to know the 10 cells, that are congested based on Radio Access Networks. Release codes are not all the time indicating real failure. Important to rely on key performance for failure. 

- With the identified congested cells, we want to identify the 10 most impacted customers in California.  


## Set up environment

### Install required packages

Run the cell below to install required Python dependencies. You may see some errors - you can safely ignore these.

In [ ]:
!pip install -r requirements.txt --quiet
!pip install langchain_experimental --quiet

### Set variables for our environment

Replace *\<values in angle brackets\>* with the correct value from your CloudFormation Outputs.

In [ ]:
AWS_REGION = "us-west-2"
ATHENA_RESULTS_BUCKET = "<ATHENA_RESULTS_BUCKET>" # Get this from CFN outputs
NETWORK_DB="network-data"

### Set up and test Bedrock access

In [ ]:

#SQLAlchemy is the Python SQL toolkit and Object Relational Mapper that gives application developers the full power and flexibility of SQL
import sqlalchemy
from sqlalchemy import create_engine

from langchain.docstore.document import Document
from langchain import PromptTemplate,SagemakerEndpoint,SQLDatabase, LLMChain

#Chain for interacting with SQL Database.
from langchain_experimental.sql import SQLDatabaseChain

#additional Lanchain modules which are being used throughout the notebook
from langchain.llms.sagemaker_endpoint import LLMContentHandler
from langchain.chains.question_answering import load_qa_chain
from langchain.prompts.prompt import PromptTemplate
from langchain_experimental.sql import SQLDatabaseSequentialChain

from langchain.chains.api.prompt import API_RESPONSE_PROMPT
from langchain.chains import APIChain
from langchain.prompts.prompt import PromptTemplate
from langchain.chat_models import ChatAnthropic
from langchain.chat_models import ChatOpenAI
from langchain.chains.api import open_meteo_docs

from typing import Dict

In [ ]:
#here we will import langhcain module which is being used with claude anthropic LLM which will we will use - hosted on Amazon Bedrock
from langchain.chat_models import ChatAnthropic as Anthropic
import anthropic
from langchain.chat_models.bedrock import BedrockChat
from langchain.llms.bedrock import Bedrock

import boto3
import json

In [ ]:
#The AWS SDK for Python (Boto3) provides a Python API for AWS infrastructure services. Using the SDK for Python, you can build applications on top of Amazon S3, Amazon EC2, Amazon DynamoDB, and more.
session = boto3.Session()
session._loader.search_paths.extend(["."])

In [ ]:
#using bedrock runtime api to invoke the claude instant model 
#further guidance : https://docs.aws.amazon.com/bedrock/latest/userguide/api-methods-run-inference.html

max_tokens_to_sample = 4096 
temperature = 0.0
stop_sequences = ["Human", "Question", "Customer", "Guru", "</s>"]

boto3_bedrock = session.client("bedrock-runtime", AWS_REGION)

json_obj = {"prompt": """
Human: you are an expert Telco business analysts
Assistant: 
""",
                "max_tokens_to_sample": max_tokens_to_sample,
                "temperature": temperature,
                "stop_sequences": stop_sequences
        }
payload = json.dumps(json_obj)
boto3_bedrock.invoke_model(
            modelId= "anthropic.claude-instant-v1",
            contentType= "application/json",
            accept= "application/json",
            body=payload)

## Set up database chain

In [ ]:
endpoint = "anthropic.claude-instant-v1"

In [ ]:
llm = BedrockChat(model_id=endpoint, client=boto3_bedrock, model_kwargs={"temperature":0.0, "max_tokens_to_sample": 4096})
#llm = Bedrock(model_id=endpoint, client=boto3_bedrock, model_kwargs={"temperature":0.0, "max_tokens_to_sample": 4096})

In [ ]:
## we will use sqlalchemy to invoke athena as our database engine for the sql queries to be executed on Amazon S3
from sqlalchemy import create_engine

## athena variables
connathena = f"athena.{AWS_REGION}.amazonaws.com"
portathena = '443'
schemaathena = NETWORK_DB

s3stagingathena=f's3://{ATHENA_RESULTS_BUCKET}/'
wkgrpathena='primary'

connection_string = f"awsathena+rest://@{connathena}:{portathena}/{schemaathena}?s3_staging_dir={s3stagingathena}/&work_group={wkgrpathena}"

##  Create the athena  SQLAlchemy engine
engine_athena = create_engine(connection_string, echo=False)

In [ ]:
engine_athena

In [ ]:
dbathena = SQLDatabase(engine_athena)
gdc = [schemaathena] 

In [ ]:
#list of tables which are exposed by the AWS Glue data catalog for athena to query
dbathena.get_usable_table_names()

In [ ]:
#we will parse the AWS glue data catalog so we can store the metadata as table info so we can push it via the prompt 
#to provide the relevant context for our model to understand which tables should  be used when constructing the SQL queries.
import boto3
def parse_catalog_tables():
    # Connect to Glue catalog
    # Get metadata of redshift serverless tables
    tables_and_columns = []
    # Define Glue client
    glue_client = boto3.client('glue')
    for db in gdc:  # Assuming gdc is a list of database names
        response = glue_client.get_tables(DatabaseName=db)
        for table in response['TableList']:
            table_name = table['Name']
            for column in table['StorageDescriptor']['Columns']:
                column_name = column['Name']
                # Remove the prefix if it exists
                if table_name.startswith("telecom_data_fabric_dbt_dan_"):
                    table_name = table_name[len("telecom_data_fabric_dbt_dan_"):]
                tables_and_columns.append(f'{table_name}|{column_name}')
    # Return the list as a newline-separated string
    return '\n'.join(tables_and_columns)
# Example usage:
gdc = [schemaathena]  # Replace with your list of database names
glue_catalog = parse_catalog_tables()
print(glue_catalog)

In [ ]:
# templates is the list of the different prompt_template you are testing. You can add several ones and select the one to use in the function chain as following chain = make_chain(query, templates[1])
templates = {
 1: """
    Given an input question, create a syntactically correct {dialect} query to run.
    Return the well formatted SQL query in the answer with the markdown format
    Only use the following tables:

    {table_info}

    Question: {input} """,
 2: """ """

}

In [ ]:
from sqldbchain import SQLDatabaseChain2

In [ ]:
#using lancghcain - we are orchestrating a flow where:
# user is generating a question->LLM translating from english to SQL -> athena triggering sql on the s3 ->athena brings back the results 
def make_chain(query, template):
    PROMPT_sql = PromptTemplate(
            input_variables=["input", "table_info", "dialect"], template=template
        )

    return SQLDatabaseChain2.from_llm(llm, dbathena, prompt=PROMPT_sql, verbose=True)
    
#define a function that infers the channel/database/table and sets the database for querying
def run_query(query, template):         
    db_chain = make_chain(query, template)
    response=db_chain.run(query=query, table_names_to_use=dbathena.get_usable_table_names())
    
    return response

## Run queries

Try the queries below by uncommenting the one you would like to try and then executing the cell sequence.

Note: You may get some errors for some queries, as the LLM we are using may not always get the SQL query right. If this happens, try executing the query again.

In [ ]:
# identify how many active users in California
query = "How many active users in California?"

# assessing what are the reasons of failures in california and expecting that the model will list and explain them in term of drop calls, white communications, etc. 
#query = "List the top 3 reasons of failures in California with their descriptions."

# from those reasons of failure, we select 38 which is about Network out of order and ask about the associated top 10 cells.
#query = "What are the top cells having release cause 38 as a release code in California?"

# now that we have the cells IDs with release 38, we want to know the 10 cells, that are congested based on Radio Access Networks. Release code are not all the time indicating real failure. Important to rely on key performance for failure. 
#query = "What are the top 10 cells that are congested in Califorina, taking into account the PRB utilisation and the RRC connection?" 

# with the identified congested cells, we want to identify the 10 most impacted customers in California.  
# query = "List the 10 most impacted MSISDN by congestion in California."

In [ ]:
# this cell is just to show you what was built within the chain and the several parameters you might need to customize for your use case
chain = make_chain(query, templates[1])
print (chain)

In [ ]:
response =  run_query(query, templates[1])